In [12]:
# https://help.github.com/articles/basic-writing-and-formatting-syntax/

# Interactive
This notebook holds interactive controls to explore inference models qualitatively. You can:
1. Create a pianoroll visualization from a NoteSequence with BokehJS.
2. Synthesize audio from a NoteSequence with the gorgeous Yamaha C5 Salamander SoundFont. You can download it too!

## Environment Setup

**JUPYTER**
* Do not use Jupyter Lab. It disables JS which breaks Bokeh and the pianoroll.
* Start a regular Jupyter Notebook with ~increased~ data rate. Otherwise the audio synthesis overflows.
    * `jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000 1.5\ Interactively\ Explore.ipynb`
    
**FILE DEPENDENCIES**
* Salamander piano SoundFont. Samples by Alexander Holm [[Link](https://archive.org/details/SalamanderGrandPianoV3)]. Converted to sf2 by John Nebauer [[Link](https://sites.google.com/site/soundfonts4u)]. You can fetch those from here (591.9 MiB):
    * `gsutil -m cp gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /tmp/`
    
**OTHER DEPENDENCIES**

To synthesize audio, you will need `fluidsynth` [[Homepage](http://www.fluidsynth.org/)] [[GitHub](https://github.com/FluidSynth/fluidsynth)] and `pyfluidsynth` [[GitHub](https://github.com/nwhitehead/pyfluidsynth)]. Be warned, those can be a pain to install. 
* `fluidsynth` failed repeatingly with `brew`. Eventually, installation worked with `macports` after I updated the entire tree (which took ages).
* `pyfluidsynth` implements Python bindings. Note that there are a few forks floating online. Some of the high ranking results on Google are dated version for Python 2.7. In the end, [this](https://github.com/nwhitehead/pyfluidsynth) worked for me.
* Keep in mind as well that there is a PyPI (`pip install`) module called `fluidsynth` which you must NOT installed.
    
    
    
Lastly, this Jupyter borrows backend from the [Magenta Performance RNN Colaboratory](https://colab.research.google.com/notebooks/magenta/performance_rnn/performance_rnn.ipynb?authuser=0#scrollTo=nzGyqJja7I0O).

## First things first

In [1]:
from magenta.music.performance_lib import PerformanceEvent
import magenta.music as mm
import re

In [2]:
interactive_config = {}
interactive_config['steps_per_quarter'] = 4

In [3]:
# Hack to allow python to pick up the newly-installed fluidsynth lib.
import ctypes.util
def proxy_find_library(lib):
    if lib == 'fluidsynth':
        return 'libfluidsynth.so.1'
    else:
        return ctypes.util.find_library(lib)

ctypes.util.find_library = proxy_find_library

In [4]:
class InteractiveSess():
    def __init__(self):
        pass
    
    def _parse_seq(self, seq):
        """
        Args:
            A text-like string of events (i.e. "ON50 ON71 SHIFT50 OFF71")
        Returns:
            The text converted into a NoteSequence class type.
        """    
        events = seq.split()
        
        performance = mm.MetricPerformance(
          steps_per_quarter=interactive_config['steps_per_quarter'])
        
        for event in events:
            if re.match(r'^ON[0-9]+$', event):
                event_type = 1         
            elif re.match(r'^OFF[0-9]+$', event):
                event_type = 2
            elif re.match(r'^SHIFT[0-9]+$', event):
                event_type = 3
            else:
                raise ValueError('Unknown event type: %s' % event)
                
            event_value = int(re.search(r'[0-9]+', event).group(0))
            
            event = PerformanceEvent(event_type, event_value)
            performance.append(event)

        note_seq = performance.to_sequence()
        return note_seq
        
    
    def load(self, path, seq_index=0):
        with open(path, 'r') as f:
            text_seqs = f.readlines()
            text_seqs = [l.strip() for l in text_seqs]
        
        text_seq = text_seqs[seq_index]
        print('INFO: Found {} sequence examples. Using seq indexed at {}. The rest are ignored.'.format(len(text_seqs), seq_index))

        note_seq = self._parse_seq(text_seq)
        
        return note_seq

In [61]:
path_to_sequence = './data/processed/heron/eval_inputs.txt'
path_to_sequence = './data/processed/heron/eval_targets.txt'

# 848
# 896
# 920 # Lion King, Circle of Life
# 2420

interactive = InteractiveSess()
sequence = interactive.load(path_to_sequence, 2420)

INFO: Found 8544 sequence examples. Using seq indexed at 2420. The rest are ignored.


## Synthesizer

Creates an interactive player for a synthesized note sequence.

In [62]:
mm.play_sequence(sequence, mm.midi_synth.fluidsynth,
                 sf2_path='./assets/Yamaha-C5-Salamander-JNv5.1.sf2')


# # Lower sound quality than `fluidsynth` with Yamaha C5 or other 
# # good SoundFont but good fallback if you don't have the SoundFont
# mm.ntebook_utils.play_sequence(sequence)

## Piano Roll Viz

In [50]:
mm.plot_sequence(sequence)
# mm.notebook_utils.plot_sequence(sequence)

Loading BokehJS ...

## MIDI

If you need to get proper MIDI, it can be useful to know the following: